### Notebook to load the model for inference

In [ ]:
# import the libraries
import sys
sys.path.append('../code/')

from components.Text_Generator import Text_Generator
from components.Question import Question
from components.Knowledge_graph import Knowledge_graph
from components.Language import Language
from components.Query import Query

from transformers import T5Tokenizer

In [ ]:
# path to model
model_path = "../fine-tuned_models/qald9plus-finetune_lcquad2-ft-base_lc-ent"
# input data attributes for model
# KG to use
knowledge_graph="Wikidata"
# to extract & utilize linguistic information or not
linguistic_context = True
# to extract & utilize entity knowledge or not
entity_knowledge = True
# Padding length for the question
question_padding_length = 128
# Padding length for the entity
entity_padding_length = 64

In [ ]:

Question.lm_tokenizer = T5Tokenizer.from_pretrained('google/mt5-xl', legacy=False)
Question.lm_tokenizer.add_tokens(["<start-of-pos-tags>", "<start-of-dependency-relation>", "<start-of-dependency-tree-depth>", "<start-of-entity-info>"])
# Initialize the model
sparql_model = Text_Generator(model_path)


In [ ]:
kg = Knowledge_graph[knowledge_graph]

def get_wikidata_entities(question: Question):
        ner = Language.get_supported_ner(question.language)
        if ner == "no_ner":
            entity_knowledge = []
        else:
            entity_knowledge = question.recognize_entities(ner, "mgenre_el")
        return entity_knowledge

def get_dbpedia_entities(question: Question):
        return question.recognize_entities("babelscape_ner" ,"mag_el")

# Function to prepare input
def prep_input(input_str, lang, linguistic_context, entity_knowledge, question_padding_length, entity_padding_length, kg):
    lang = Language(lang)
    question = Question(input_str, lang)
    question_string = question.question_string
    if linguistic_context:
        question_string = question.get_question_string_with_lingtuistic_context(question_padding_length)
    if entity_padding_length:
        if kg==Knowledge_graph.Wikidata:
            entity_knowledge = get_wikidata_entities(question)
        elif kg==Knowledge_graph.DBpedia:
            entity_knowledge = get_dbpedia_entities(question)
    question_string = question.add_entity_knowledge(question_string, entity_knowledge, entity_padding_length)
    return question_string


In [ ]:
lang = "en"
question_str = "Give me the grandchildren of Bruce Lee"
processed_question_string = prep_input(question_str, lang, linguistic_context, entity_knowledge, question_padding_length, entity_padding_length, kg)
print(processed_question_string)

In [ ]:

pred_sparql = sparql_model.predict_sparql(processed_question_string)
query = Query(pred_sparql, kg)
print(query.sparql)